In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import collections
import copy
import sys
sys.stdout.flush()

from processing_functions import *

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'dejavuserif'
rcParams['mathtext.rm'] = 'serif'
rcParams['mathtext.it'] = 'serif:italic'
rcParams['mathtext.bf'] = 'serif:bold'
rcParams['axes.titlepad'] = 12

In [2]:
jkcov = np.load('Run_30_bins_for_all/jk_19p0.npz')['jkcov']
fidreal = np.load('Run_30_bins_for_all/fidreal_19p0.npz')['func']
#fidrandom = np.load('Run_30_bins_for_all/')

In [3]:
covtot = calc_covariance(jkcov,fidreal,0)

TypeError: 'int' object has no attribute '__getitem__'